In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler


In [2]:

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the dataset
df = pd.read_csv('creditcard.csv')
X = df.iloc[:, 1:-1].values
y = df.iloc[:, -1].values


Using device: cuda


In [3]:

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

# Convert data to PyTorch tensors and move to GPU
X = torch.tensor(X, dtype=torch.float32).to(device)

# SOM parameters
map_size = (10, 10)  # SOM grid size
input_len = X.shape[1]  # Number of features
sigma = 1.0  # Neighborhood radius
learning_rate = 0.5  # Learning rate
epochs = 5  # Number of training epochs


In [4]:

# Initialize SOM weights
som_weights = torch.randn(map_size[0], map_size[1], input_len, device=device)

# Training function
def train_som(X, som_weights, epochs, sigma, learning_rate):
    for epoch in range(epochs):
        for sample in X:
            # Find the Best Matching Unit (BMU)
            distances = torch.sum((som_weights - sample) ** 2, dim=2)
            bmu_index = torch.argmin(distances)
            bmu = (bmu_index // map_size[1], bmu_index % map_size[1])

            # Update weights
            for i in range(map_size[0]):
                for j in range(map_size[1]):
                    # Calculate the neighborhood function
                    distance_to_bmu = torch.sqrt((i - bmu[0]) ** 2 + (j - bmu[1]) ** 2)
                    neighborhood = torch.exp(-distance_to_bmu ** 2 / (2 * sigma ** 2))

                    # Update the weight
                    som_weights[i, j] += learning_rate * neighborhood * (sample - som_weights[i, j])

        # Decay learning rate and sigma
        learning_rate *= 0.99
        sigma *= 0.99

    return som_weights


In [5]:

# Train the SOM
som_weights = train_som(X, som_weights, epochs, sigma, learning_rate)

# Move SOM weights back to CPU for visualization
som_weights = som_weights.cpu()

# Visualization
def plot_som(som_weights):
    plt.figure(figsize=(10, 10))
    plt.pcolor(som_weights.mean(dim=2).numpy(), cmap='bone')
    plt.colorbar()
    plt.show()

plot_som(som_weights)

KeyboardInterrupt: 